In [ ]:
import time
import pandas as pd
import awswrangler as wr
from datetime import datetime

In [ ]:
today = datetime.now().strftime('%Y-%m-%d')
s3_bucket = "amazon-forecast-blood-analysis"
athena_table = "blood_blood_analysis"
athena_database = "blood_analisys"

# Realizar Query no Athena com o wr e buscar os dados dos d - 30 enviados pelo IoT Core
# Realizar resample dos dados que sao coletados de 5 min em 5 min para hora em hora
# Subir esses dados para o s3 criando o catalog com o WR

# TODO: alterar query para D - 30
# df_read_athena = wr.athena.read_sql_query(f"SELECT * FROM {athena_table}", database=athena_database)
blood_five_minutes = df_read_athena.sort_values(by='timestamp',ascending=True)
blood_five_minutes = blood_five_minutes.dropna()

unique_ids = blood_five_minutes['user_id'].unique() # Get Unique ids to iterate
appended_data = []

for ids in unique_ids:
    spplitted_data = blood_five_minutes.loc[blood_five_minutes['user_id'] == str(ids)]

    # Fazer resample convertendo de 5 em 5 min para dados de 1 em 1 hora.
    data_series = spplitted_data.set_index('timestamp')
    data_series.index = pd.to_datetime(data_series.index)
    data_series = data_series.resample('1H').bfill()
    
    blood_analysis_hourly = data_series.reset_index()
    blood_analysis_hourly['timestamp'] = blood_analysis_hourly['timestamp'].dt.strftime('%Y-%m-%d %H:%m:%S')
    blood_analysis_hourly = blood_analysis_hourly.drop(['partition_0'], axis=1)
    blood_analysis_hourly = blood_analysis_hourly.drop(['insulina'], axis=1) # Removendo insulina
    blood_analysis_hourly = blood_analysis_hourly.drop(['carbo'], axis=1) # Removendo carboidrato
    
    appended_data.append(blood_analysis_hourly)

concatenated_df = pd.concat(appended_data, ignore_index=True)

s3_path_glucose = f"s3://{s3_bucket}/stage/blood_analysis/glucose/{today}/"
s3_path_df = {s3_path_glucose : {'df_name' : concatenated_df, 'table_name' : 'blood_analysis_stage_hourly'}}

for s3_path, values in s3_path_df.items():
    wr.s3.to_csv(
        df=values['df_name'],
        path=s3_path,
        index=False,
        dataset=True,
        mode="overwrite",
        database="blood_analisys",
        table=values['table_name']
        )

